In [13]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.svm import SVR


In [14]:
properties = pd.read_csv('/home/santiago/Desktop/Organizacion De Datos (75.06)/TP2/trainFull.csv', low_memory=False)
test_set = pd.read_csv('/home/santiago/Desktop/Organizacion De Datos (75.06)/TP2/testFull.csv', low_memory=False)

In [15]:
#train = properties.loc[:, ['property_type_encoded', 'place_name_encoded', 'rooms'
 #                           , 'surface_total_in_m2','price']]

#train = properties.loc[:, ['acceso','agua','aire','amplio','balc','price']]

properties['lat'] = properties['lat'].apply(lambda x: x*2000)
properties['lon'] = properties['lon'].apply(lambda x: x*2000)

train = properties.loc[:, (properties.columns != 'id') & (properties.columns != 'created_on') & 
                       (properties.columns != 'description')]

    
train = train[:int(0.2*len(properties))]# Tomo un 20% del set de entrenamiento, dado que svr solo depende
#de la cantidad de support vector
train_prices = properties[['price']][:int(0.2*len(properties))]


train_to_predict = properties[int(0.2*len(properties)):]
#train_to_predict = train_to_predict.loc[:, ['property_type_encoded', 'place_name_encoded', 'rooms'
 #                                           , 'surface_total_in_m2','price']]
#train_to_predict = train_to_predict.loc[:, ['acceso','agua','aire','amplio','balc','price']]
train_to_predict = train_to_predict.loc[:, (train_to_predict.columns != 'id') & (train_to_predict.columns != 'created_on') & 
                       (train_to_predict.columns != 'description')]
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 66 columns):
Unnamed: 0                  14166 non-null int64
id                          14166 non-null int64
created_on                  14166 non-null object
property_type_encoded       14166 non-null int64
place_name_encoded          14166 non-null int64
state_name_encoded          14166 non-null int64
lat                         14166 non-null float64
lon                         14166 non-null float64
surface_covered_in_m2       14166 non-null float64
surface_total_in_m2         14166 non-null float64
floor                       14166 non-null float64
rooms                       14166 non-null float64
distancia_obelisco          14166 non-null float64
distancia_minima_subte      14166 non-null float64
description                 14166 non-null object
distancia_minima_privado    14166 non-null float64
distancia_minima_publico    14166 non-null float64
acceso                      14166 no

In [16]:
#Ahora si empieza K-Means
#C es la penalizacion por clasificar mal (uso soft margins)
#gamma es el sigma del kernel gaussiano
#rbf implica que estoy usando un kernel gaussiano
svr = SVR(kernel = 'rbf', gamma = 0.1, C = 1)
predictions = svr.fit(train,train_prices.values.ravel()).predict(train_to_predict)
predictions = predictions.flatten()


In [17]:
result = pd.DataFrame()
result['price_usd'] = predictions
result['price'] = train_to_predict['price'].values
score = mean_squared_error(result['price'].values, result['price_usd'])
score


660339529948.52795

In [18]:
result

,price_usd,price
0,140000.230464,195000.0
1,140000.230464,345000.0
2,140000.230464,480000.0
3,140000.230464,130000.0
4,140000.230464,330000.0
5,140000.230464,73000.0
6,140000.230464,107000.0
7,140000.230464,125000.0
8,140000.230464,200000.0
9,140000.230464,200000.0


In [19]:
'''
c_val = 0.1
score = 0
gamma_val = 0.1
best_score = 0
best_gamma = 0
best_c = 0
first = True

while c_val < 100:
    while gamma_val < 100:
        svr = SVR(kernel = 'rbf', gamma = gamma_val, C = c_val)
        predictions = svr.fit(train,train_prices.values.ravel()).predict(train_to_predict)
        predictions = predictions.flatten()
        score = mean_squared_error(predictions, train_to_predict['price'].values)
        if (score < best_score) | (first):
            first = False
            best_score = score
            best_gamma = gamma_val
            best_c = c_val
        c_val+= 0.1
        gamma_val+= 0.1
        
print('best_Score: ' + str(score))
print('C_best: '  + str(best_c))
print('gamma_best: ' + str(best_gamma))
'''

"\nc_val = 0.1\nscore = 0\ngamma_val = 0.1\nbest_score = 0\nbest_gamma = 0\nbest_c = 0\nfirst = True\n\nwhile c_val < 100:\n    while gamma_val < 100:\n        svr = SVR(kernel = 'rbf', gamma = gamma_val, C = c_val)\n        predictions = svr.fit(train,train_prices.values.ravel()).predict(train_to_predict)\n        predictions = predictions.flatten()\n        score = mean_squared_error(predictions, train_to_predict['price'].values)\n        if (score < best_score) | (first):\n            first = False\n            best_score = score\n            best_gamma = gamma_val\n            best_c = c_val\n        c_val+= 0.1\n        gamma_val+= 0.1\n        \nprint('best_Score: ' + str(score))\nprint('C_best: '  + str(best_c))\nprint('gamma_best: ' + str(best_gamma))\n"